In [1]:
import boto3
import weaviate
import weaviate.classes as wvc

# Initialize AWS clients
ecs_client = boto3.client('ecs')
ec2_client = boto3.client('ec2')

# Get the task ARN
response = ecs_client.list_tasks(cluster='weaviate-cluster', serviceName='weaviate-service')
task_arn = response['taskArns'][0]

# Get the ENI ID
response = ecs_client.describe_tasks(cluster='weaviate-cluster', tasks=[task_arn])
eni_id = next(
    detail['value']
    for task in response['tasks']
    for attachment in task['attachments']
    for detail in attachment['details']
    if detail['name'] == 'networkInterfaceId'
)

# Get the Weaviate IP
response = ec2_client.describe_network_interfaces(NetworkInterfaceIds=[eni_id])
WEAVIATE_IP = response['NetworkInterfaces'][0]['Association']['PublicIp']

print(f"Weaviate IP: {WEAVIATE_IP}")
%store WEAVIATE_IP

# Connect to Weaviate
client = weaviate.connect_to_local(WEAVIATE_IP)

collection = client.collections.create(
    name="TestCollection",
    properties=[
        wvc.config.Property(
            name="title",
            data_type=wvc.config.DataType.TEXT
        )
    ]
)
print("Created collection.")
collections = client.collections.list_all()
client.collections.delete("TestCollection")
print(collections)
client.close()

/opt/conda/lib/python3.12/site-packages/botocore/auth.py:425: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow()


Weaviate IP: 44.244.93.188
Stored 'WEAVIATE_IP' (str)
Created collection.
{'TestCollection': _CollectionConfigSimple(name='TestCollection', description=None, generative_config=None, properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={})], references=[], reranker_config=None, vectorizer_config=None, vectorizer=None, vector_config={'default': _NamedVectorConfig(vectorizer=_NamedVectorizerConfig(vectorizer=<Vectorizers.NONE: 'none'>, model={}, source_properties=None), vector_index_config=_VectorIndexConfigHNSW(multi_vector=None, quantizer=None, cleanup_interval_seconds=300, distance_metric=<VectorDistances.COSINE: 'cosine'>, dynamic_ef_min=100, dynamic_ef_max=500, dynamic_ef_factor=8, ef=-1, ef_construction=128, filter_strategy=<VectorFilterStrategy.SWEEPI